Installation of Tensorflow

In [4]:
!pip install tensorflow-gpu==2.0.0-beta1

import tensorflow as tf

     |████████████████████████████████| 348.9MB 44kB/s 
     |████████████████████████████████| 501kB 54.3MB/s 
     |████████████████████████████████| 3.1MB 52.5MB/s 


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:


print(tf.__version__)

2.0.0-beta1


In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Input


Drive has to be mounted so the dataset is available for the model.

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [7]:
%cd '/content/gdrive/My Drive/SIH'

/content/gdrive/My Drive/SIH


In [0]:
#weights_path = 'resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'


MobileNet v2 

In [8]:
"""num_classes = 2

test_model = Sequential()
test_model.add(ResNet50(include_top=False,
                        pooling='avg',
                        input_shape = (224, 224, 3),
                        weights = 'imagenet'))
test_model.add(Dense(num_classes, activation='softmax'))
test_model.layers[0].trainable = False

test_model.compile(optimizer='sgd', loss= 'categorical_crossentropy', metrics= ['accuracy'])"""

base_model = MobileNetV2(input_shape = (224, 224, 3),
                         weights = "imagenet",
                         include_top = False)



KeyboardInterrupt: ignored

In [0]:
base_model.trainable = False

Architecture of the model

In [0]:
base_model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________

In [9]:
# Flattening
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

#Classifying layer
prediction_layer = tf.keras.layers.Dense(1)

#Connecting all layers
model = tf.keras.Sequential([base_model,
                             global_average_layer,
                             prediction_layer])
base_learning_rate = 0.0001

model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input

In [0]:
#Image datagenerator for data augementation and creating the dataset which can be fed to the model.
#We just have to make the directories according  to their classes.
#Makes our job very easy

data_generator = ImageDataGenerator(preprocessing_function=preprocess_input, rescale=1./255)


In [0]:
%ls

dataset/  Pothole.ipynb


In [15]:
!pip install -q pyyaml h5py 

KeyboardInterrupt: ignored

In [44]:
training_set = data_generator.flow_from_directory(
        'Dataset/training_set',
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary')

test_set =  data_generator.flow_from_directory(
        'Dataset/test_set',
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary')

print(training_set.class_indices)





Found 520 images belonging to 2 classes.
Found 126 images belonging to 2 classes.
{'non-pothole': 0, 'pothole': 1}


In [0]:
new_model = tf.keras.models.load_model('training1.h5')

In [32]:
new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 1281      
Total params: 2,259,265
Trainable params: 1,281
Non-trainable params: 2,257,984
_________________________________________________________________


In [40]:
%ls


Dataset/                                              Test/
Pothole.ipynb                                         training1.h5
resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5


In [13]:
new_model.class_indices

NameError: ignored

In [12]:
import numpy as np
import os

dir = 'Test/pothole'

files = os.listdir(dir)

d = {}

for file in files:
  d[file] = 1





{'n293.jpg': 1,
 'n294.jpg': 1,
 'n444.jpg': 1,
 'nn492.jpg': 1,
 'nn494.jpg': 1,
 'nn495.jpg': 1,
 'nn496.jpg': 1,
 'nn499.jpg': 1}

In [0]:
dir = 'Test/non-pothole'

files = os.listdir(dir)

for file in files:
  d[file] = 0

In [24]:
d

{'n12.jpg': 0,
 'n14.jpg': 0,
 'n293.jpg': 1,
 'n294.jpg': 1,
 'n431.jpg': 0,
 'n438.jpg': 0,
 'n442.jpg': 0,
 'n444.jpg': 1,
 'nn410.jpg': 0,
 'nn412.jpg': 0,
 'nn413.jpg': 0,
 'nn492.jpg': 1,
 'nn494.jpg': 1,
 'nn495.jpg': 1,
 'nn496.jpg': 1,
 'nn499.jpg': 1}

In [0]:
n

In [48]:
import numpy as np
from keras.preprocessing import image

img = image.load_img('Test/n12.jpg', target_size = (224, 224))
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
result = new_model.predict(img)

print(result[0][0])


"""
k = list(d.keys())
print(k)

size = len(k)

cor = None

for i in k:
  image_name = 'Test/' + i
  img = image.load_img(image_name, target_size=(224, 224))
  img = image.img_to_array(img)
  img = np.expand_dims(img, axis=0)
  result = new_model.predict(img)
  print(i, end= ' ')
  if d[i] == 0:
    print('Non-Pothole')
  else:
    print('Pothole')
  print('Predicted : ' , end = " ")
  if result == 0 :
    print('Non-Pothole')
  else:
    print('Pothole')
  if(d[i] == result):
    cor += 1

acc = cor/size
print(acc)"""


-3.2914553


'\nk = list(d.keys())\nprint(k)\n\nsize = len(k)\n\ncor = None\n\nfor i in k:\n  image_name = \'Test/\' + i\n  img = image.load_img(image_name, target_size=(224, 224))\n  img = image.img_to_array(img)\n  img = np.expand_dims(img, axis=0)\n  result = new_model.predict(img)\n  print(i, end= \' \')\n  if d[i] == 0:\n    print(\'Non-Pothole\')\n  else:\n    print(\'Pothole\')\n  print(\'Predicted : \' , end = " ")\n  if result == 0 :\n    print(\'Non-Pothole\')\n  else:\n    print(\'Pothole\')\n  if(d[i] == result):\n    cor += 1\n\nacc = cor/size\nprint(acc)'